# Text-Klassifikation (Aufgabe s.u.)
Das Beispiel basiert auf einem offenen Datensatz von Newsgroup-Nachrichten und orientiert sich am offiziellen Tutorial von scikit-learn zur Textanalyse.

Wir nutzen Dokumente von mehreren Newsgroups und trainieren damit einen Classifier, der dann eine Zudordnung von neuen Texten auf eine dieser Gruppen durchführen kann. Sprich die Newsgroups stellen die Klassen/Tags dar, mit denen wir neue Texte klassifizieren. Wie nutzen einen einfachen Bag-of-Word-Ansatz in dem wir (normalisierte) Häufigkeit von Wörtern als Features nutzen

In [1]:
# Daten laden
# Daten sind kein Teil von scikit-learn, Daten laden über eine gesonderte Funktion
from sklearn.datasets import fetch_20newsgroups

In [2]:
# Wir legen vier Newsgroups fest, die wir nutzen wollen.
selected_categories = ["sci.crypt", "sci.electronics", "sci.med", "sci.space"]

In [3]:
# Wir beziehen die Trainingset- und Testsets-Dokumente.
newsgroup_posts_train = fetch_20newsgroups(
    data_home="newsgroup_data",
    subset='train',
    categories=selected_categories,
    shuffle=True, random_state=1)
newsgroup_posts_test = fetch_20newsgroups(
    data_home="newsgroup_data",
    subset='test',
    categories=selected_categories,
    shuffle=True, random_state=1)

In [4]:
# Die Objekte, die wir erhalten, sind scikit-learn-Bunches
type(newsgroup_posts_train)

sklearn.utils._bunch.Bunch

In [5]:
# haben die üblichen Attribute von Bunches
dir(newsgroup_posts_train)

['DESCR', 'data', 'filenames', 'target', 'target_names']

In [6]:
# U.a. existiert die übliche Beschreibung des Datensets im Attribut DESCR, die wir uns ansehen können.
print(newsgroup_posts_train.DESCR)

.. _20newsgroups_dataset:

The 20 newsgroups text dataset
------------------------------

The 20 newsgroups dataset comprises around 18000 newsgroups posts on
20 topics split in two subsets: one for training (or development)
and the other one for testing (or for performance evaluation). The split
between the train and test set is based upon a messages posted before
and after a specific date.

This module contains two loaders. The first one,
:func:`sklearn.datasets.fetch_20newsgroups`,
returns a list of the raw texts that can be fed to text feature
extractors such as :class:`~sklearn.feature_extraction.text.CountVectorizer`
with custom parameters so as to extract feature vectors.
The second one, :func:`sklearn.datasets.fetch_20newsgroups_vectorized`,
returns ready-to-use features, i.e., it is not necessary to use a feature
extractor.

**Data Set Characteristics:**

    =================   ==========
    Classes                     20
    Samples total            18846
    Dimensionality

In [7]:
# Das Attribut data enthält in diesem Fall keine Matrix, sondern die Daten sind Newsgroup-Messages
# Beispiel
print(newsgroup_posts_train.data[6])

From: pmetzger@snark.shearson.com (Perry E. Metzger)
Subject: Do we need the clipper for cheap security?
Organization: Partnership for an America Free Drug
Lines: 53

amanda@intercon.com (Amanda Walker) writes:
>> The answer seems obvious to me, they wouldn't.  There is other hardware 
>> out there not compromised.  DES as an example (triple DES as a better 
>> one.) 
>
>So, where can I buy a DES-encrypted cellular phone?  How much does it cost?
>Personally, Cylink stuff is out of my budget for personal use :)...

If the Clipper chip can do cheap crypto for the masses, obviously one
could do the same thing WITHOUT building in back doors.

Indeed, even without special engineering, you can construct a good
system right now. A standard codec chip, a chip to do vocoding, a DES
chip, a V32bis integrated modem module, and a small processor to do
glue work, are all you need to have a secure phone. You can dump one
or more of the above if you have a fast processor. With integration,
you could 

In [8]:
# Die Targets sind die Newsgroup-Namen.
# Diese Klassen sind wie üblich für scikit-learn als Zahlen kodiert, die wir mittels target_names auflösen können
print(newsgroup_posts_train.target_names)

['sci.crypt', 'sci.electronics', 'sci.med', 'sci.space']


In [9]:
# Für unsere Beispiel-Message:
newsgroup_posts_train.target_names[newsgroup_posts_train.target[6]]

'sci.crypt'

In [10]:
# Um die Wörter zu zählen, aber auch um Stopwörte zu entfernen und zu Tokenisieren
# nutzen wir ein Objekt der CountVectorizer-Klasse bzw. dessen fit-Methode
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
count_vect.fit(newsgroup_posts_train.data)

CountVectorizer()

In [11]:
# Über alle Dokumente bekommen wir die folgende Zusammenstellung der Wörter und ihre Indices (positionen im Array)
len(count_vect.get_feature_names_out())

38683

In [12]:
# Wir können uns ein paar Beispiele ansehen …
count_vect.get_feature_names_out()[10000:10050]

array(['cellar', 'cellphone', 'cells', 'cellsat', 'cellular', 'cellulars',
       'celluloid', 'celp', 'celsius', 'cement', 'cen', 'censoring',
       'censorship', 'censure', 'census', 'cent', 'centaur', 'centauri',
       'centaurs', 'centennial', 'center', 'centered', 'centerline',
       'centerpiece', 'centers', 'centigrade', 'centimeter',
       'centimeters', 'central', 'centralia', 'centralised', 'centralism',
       'centralization', 'centralize', 'centralized', 'centrally',
       'centre', 'centres', 'centrifuge', 'centronic', 'cents', 'centure',
       'centuries', 'century', 'ceo', 'cepek', 'cephalopods', 'cept',
       'ceramic', 'cereal'], dtype=object)

In [13]:
# … oder sogar das counting-Dictionary mit den Wörtern und ihre Vorkommen-Anzahl betrachten
# Achtung: groß!
# print(count_vect.vocabulary_)

In [14]:
# Diese Countings müssen wir für den Klassifikator in eine Matrix transformieren
X_train_counts = count_vect.transform(newsgroup_posts_train.data)

In [15]:
# Die Matrix, die wir erhalten, hat folgende Maße
X_train_counts.shape

(2373, 38683)

In [16]:
# Wir normalisieren die Wörtercoutings auf die Anzahl an Wörter im Text (Term Frequency - TF).
# Dazu nutzen wir eine Objekt der Klasse TfidfTransformer
# (schalten die idf-Normalisierung (Inverse Document Frequency) dabei ab)
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False)

In [17]:
# Die Normalisierung erfolgt mit den Methoden fit und transform
tf_transformer.fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)

In [18]:
# Die Matrix, die wir erhalten, hat folgende Maße
X_train_tf.shape

(2373, 38683)

In [19]:
# Jetzt können wir einen Klassifikator erstellen.
# Wir Nutzen hier eine Random-Forest-Klassifikator, könnten aber auch eine andere Methode wählen
from sklearn.ensemble import RandomForestClassifier
tf_random_forest_classifier = RandomForestClassifier()

In [20]:
# Wie bei allen Supervised-Learning-Verfahren trainieren wir den Klassifikator mit der Trainingsmatrix
tf_random_forest_classifier.fit(X_train_tf, newsgroup_posts_train.target)

RandomForestClassifier()

In [21]:
# Um zu testen wie gut der Klassifikator funktioniert, prozessieren wir das Test-Set mit dem CountVectorizer-Objekt
# und führen die gleiche TF-Transformation durch
X_test_counts = count_vect.transform(newsgroup_posts_test.data)
X_test_tf = tf_transformer.transform(X_test_counts)

In [22]:
# Ein kurze Blick auf die Maße der Matrix zeigt uns, dass die Anzahl
# an Spalten (Features) gleich ist wie bei der Trainingsmatrix
X_test_counts.shape

(1579, 38683)

In [23]:
# Jetzt können wir mit der score-Methods die Güte des Klassikators auf dem Test-Set prüfen
tf_random_forest_classifier.score(X_test_tf, newsgroup_posts_test.target)

0.8543381887270425

In [24]:
# Der Klassifikator scheint gut genug zu funktionieren. Wir können jetzt Listen von Dokumenten klassifizieren.
# Wir nehmen zwei Dokumete aus unserem Test-Set und erstellen zusätzlich ein sehr kleines eigene Dokument,
# das nur aus einem Satz besteht.
docs_to_classify = [
    newsgroup_posts_test.data[1],
    newsgroup_posts_test.data[7],
    "The sun send a lot of radiation to the planets including earth"]

In [25]:
# Werfen wir einen kurzen Blick auf die zwei Dokumente aus dem Testset.
print(newsgroup_posts_test.data[1])
# print(newsgroup_posts_test.data[7])

From: dmuntz@quip.eecs.umich.edu (Dan Muntz)
Subject: Re: new encryption
Organization: University of Michigan EECS Dept., Ann Arbor
Lines: 13

In article <strnlghtC5wC3z.Erw@netcom.com> strnlght@netcom.com (David Sternlight) writes:
>psionic@wam.umd.edu, whose parenthesized name is either an unfortunate
>coincidence or casts serious doubt on his bona fides, posts a message in
>which he seems willing to take the word of a private firm about which he
>knows little that their new encryption algorithm is secure and contains no
>trapdoors, while seemingly distrusting that of the government about clipper.

Will someone please post the David Sternlight FAQ to alt.privacy.clipper before
someone unfamiliar with him takes him seriously and starts yet another
flame fest?

  -Dan




In [26]:
# Auch diese neu zu klassifizierenden Dokumente müssen wir wie die Traininsdokumente in Matrizen transformieren
X_to_classify_counts = count_vect.transform(docs_to_classify)
X_to_classify_tf = tf_transformer.transform(X_to_classify_counts)

In [27]:
# Jetzt können wir mit dieser Matrix die Klassifikation durchführen …
predicted_classes = tf_random_forest_classifier.predict(X_to_classify_tf)

In [28]:
# … und uns die Klassen anschauen, mit denen die Dokumente versehen wurden.
for predicted_class in predicted_classes:
    print(newsgroup_posts_train.target_names[predicted_class])

sci.crypt
sci.med
sci.electronics


In [29]:
# Um den Klassifikator zu verbessern, testen wir statt der Term-Frequenz nun die
# TFIDF (Term Frequency times Inverse Document Frequency) und erstellen damit unsere Matrizen.
tfidf_transformer = TfidfTransformer(use_idf=True).fit(X_train_counts)

### Aufgabe 
Mach mit diesem TFIDF-Ansatz äquivalent zu der Klassifikation mit dem TF-Ansatz weiter.
D.h. führe alle nötigen Schritte wie Training, Scoring und Prediction durch.
Ist das Ergebnis besser? Gerne kannst Du zusätzlich mit anderen Klassifikator-Typen anstelle von Randeom Forest
experimentieren (z.B. SVMs oder neuronale Netzen), um zu testen, ob dies zu einer besseren Klassifikation führt.

In [30]:
# Modell wird trainiert (?)
X_train_tfidf = tfidf_transformer.transform(X_train_counts)

In [34]:
# testen wie gut der Klassifikator funktioniert
# dazu Test-Set mit dem CountVectorizer-Objekt porzessieren TFIDF-Transformation durchführen
X_test_counts = count_vect.transform(newsgroup_posts_test.data)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

In [35]:
# Random-Forest-Klassifikator wird gewählt
tfidf_random_forest_classifier = RandomForestClassifier()

# Klassifikator wird mit Trainingsmatrix trainiert
tfidf_random_forest_classifier.fit(X_train_tfidf, newsgroup_posts_train.target)

RandomForestClassifier()

In [36]:
# Güte des Modells wird berechnet
tfidf_random_forest_classifier.score(X_test_tfidf, newsgroup_posts_test.target)

0.8556048131728943

In [ ]:
# Lösung aus Musterlösung
# Das Ergebnis ist zur Klassifikation mit TF nur minimal anders und weicht von Lösungs-Ergebnis ab. Fehler?!